In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import cv2  
from tqdm import tqdm
from random import randint
import json
import matplotlib.image as mpimg
from matplotlib import rcParams
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
from keras.preprocessing.image import load_img 
from keras.preprocessing.image import img_to_array 
from tensorflow.keras.applications.vgg16 import preprocess_input,VGG16

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

Using TensorFlow backend.


In [2]:
train_path = "data/train"
test_path = "data/test"

In [3]:
train = os.listdir(train_path)
test = os.listdir(test_path)

In [4]:
train_image = [cv2.resize(cv2.cvtColor(cv2.imread(os.path.join(train_path, image)), cv2.COLOR_BGR2RGB),(150,150))  for image in train ]

In [8]:
test_image = [cv2.resize(cv2.cvtColor(cv2.imread(os.path.join(test_path, image)), cv2.COLOR_BGR2RGB),(150,150))  for image in test ]

In [5]:
label = pd.read_csv("data/train_features.csv")
label[["x","y"]]

,x,y
0,-9.380678,3.58272
1,-9.380678,3.58272
2,-9.380678,3.58272
3,-9.380678,3.58272
4,-9.380678,3.58272
...,...,...
7495,-4.680678,35.18272
7496,-4.680678,35.18272
7497,-4.680678,35.18272
7498,-4.680678,35.18272


In [6]:
y_label = []
for index, row in label.iterrows():
    y_label.append((row["x"],row["y"]))
y_label = np.array(y_label)
train_image = np.array(train_image)

In [9]:
test_image = np.array(test_image)

In [10]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1


In [11]:
vgg16 = VGG16(weights='imagenet',input_shape=[150,150,3],include_top=False)
#vgg19 = VGG19(weights='imagenet',input_shape=[150,150,3],include_top=False)
#resnet50 = ResNet50(weights='imagenet',input_shape=[150,150,3],include_top=False)

In [12]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.layers.merge import concatenate
from tensorflow.keras.models import Model

In [13]:
def train_model_full(pre_train):
  for layer in pre_train.layers:
    layer.trainable = False
  layer = layers.Conv2D(filters = 32,kernel_size = (1,1),padding = "same",activation = "relu")(pre_train.layers[-1].output)
  layer = layers.MaxPooling2D(pool_size=(2,2), strides=2,padding="same")(layer)
  layer = layers.Conv2D(filters = 64,kernel_size = (3,3),padding = "same",activation = "relu")(layer)
  layer = layers.Flatten()(layer)
  layer = layers.Dense(1024,activation="relu")(layer)
  layer = layers.Dense(2,activation="linear")(layer)

  model = tf.keras.Model(inputs = pre_train.inputs,outputs = layer)
  
  return model

In [14]:
opt = keras.optimizers.Adam(learning_rate=0.001)


In [15]:
model_vgg16_full = train_model_full(vgg16)

model_vgg16_full.compile(optimizer=opt,
            loss=tf.keras.losses.MeanAbsoluteError())

In [16]:
model_vgg16_full.fit(train_image, label[["x","y"]], epochs=50, batch_size=256)

Epoch 1/50
30/30 [==============================] - 47s 810ms/step - loss: 36.4541
Epoch 2/50
30/30 [==============================] - 17s 573ms/step - loss: 29.7609
Epoch 3/50
30/30 [==============================] - 17s 573ms/step - loss: 27.2899
Epoch 4/50
30/30 [==============================] - 17s 576ms/step - loss: 25.6271
Epoch 5/50
30/30 [==============================] - 17s 576ms/step - loss: 24.2657
Epoch 6/50
30/30 [==============================] - 17s 578ms/step - loss: 22.8883
Epoch 7/50
30/30 [==============================] - 22s 735ms/step - loss: 21.9194
Epoch 8/50
30/30 [==============================] - 23s 755ms/step - loss: 21.1325
Epoch 9/50
30/30 [==============================] - 23s 767ms/step - loss: 20.2108
Epoch 10/50
30/30 [==============================] - 23s 774ms/step - loss: 19.5074
Epoch 11/50
30/30 [==============================] - 23s 781ms/step - loss: 18.8432
Epoch 12/50
30/30 [==============================] - 21s 708ms/step - loss: 18.1227
E

In [17]:
result = model_vgg16_full.predict(test_image)


In [18]:
x = []
y = []
for i in result:
   x.append(i[0])
   y.append(i[1]) 

In [19]:
test_result = pd.read_csv("data/imagenames.csv")

In [20]:
test_result["x"] = x
test_result["y"] = y

In [ ]:
test_result.to_csv("result/CNN_baseline_vgg16.csv",index=False)